In [9]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import uniform
from scipy.stats import norm
import pandas as pd
%matplotlib inline

#global LIST_OF_BOXES
#LIST_OF_BOXES = []

/home/kondrat/.conda/envs/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [10]:
def print_d(d):
    d_ = [{'probability':p, 'reward':r} for r,p in d.items()]
    return pd.DataFrame(d_)

In [11]:
def calc_stats(d_of_proba):
    assert sum(d_of_proba.values()) <= 1
    assert all(np.array(list(d_of_proba.values())) > 0)
    
    E = 0
    V = 0
    for r, p in d_of_proba.items():
        E+=p*r
    
    for r, p in d_of_proba.items():
        V+=p*((r-E)**2)
    
    V+=(1-sum(d_of_proba.values()))*(E**2)
    return E,V

def make_d(d_of_proba: dict):
    assert sum(d_of_proba.values()) <= 1
    assert all(np.array(list(d_of_proba.values())) > 0)
    
    d = {}
    a = 0
    for r, p in d_of_proba.items():
        d[(a,a+p)] = r
        a+=p

    return d

def make_dist_func(d_of_proba:dict):
    d = make_d(d_of_proba)
    check_in = lambda x,b: x>=b[0] and x<b[1]
    
    def f(x):
        for b, reward in d.items():
            if check_in(x,b):
                return reward
        else:
            return 0
    
    return f

In [26]:
class box():
    def __init__(self, d, cost):
        self.d = d
        self.E, self.V = calc_stats(d)
        
        self.cost = cost
        self.spin_f = make_dist_func(d)
        
        self.total_win = 0
        self.total_loss = 0
        self.spins_count = 0
        self.spins_result = np.array([])
        #LIST_OF_BOXES.append(self)
        
    def spin(self, count=1):
        self.total_loss+=count*self.cost
        self.spins_count+=count
        result = np.array([self.spin_f(x) for x in uniform(size=count)])
        self.total_win+=result.sum()
        self.spins_result = np.hstack([self.spins_result,result])
        
        return result
    
    def reset(self):
        self.total_win = 0
        self.total_loss = 0
        self.spins_count = 0
        self.spins_result = np.array([])
    
    def stats(self, full=False):
        if full:
            print(f'E : {self.E}')
            print(f'V: {self.V}')
            print(f'cost: {self.cost}')
            print(f'sum mean: {self.E - self.cost}')
            print('------------')
        print(f'total win: {self.total_win}')
        print(f'total loss: {self.total_loss}')
        print(f'total profit: {self.total_win - self.total_loss}')
        print(f'spins: {self.spins_count}')
        if self.spins_count:
            print(f'mean profit: {(self.total_win - self.total_loss)/self.spins_count}')
            print('##############')
            rs, cs = np.unique(self.spins_result, return_counts=True)
            for r,c in zip(rs,cs):
                print(f'reward {r}: {c}')

In [13]:
def calc_boxes_stats(list_of_box):
    E = 0
    V = 0
    N = 0
    total_win = 0
    total_loss = 0
    mean_cost = 0
    for box in list_of_box:
        E+=box.E*box.spins_count
        V+=box.V*box.spins_count
        N+=box.spins_count
        total_win+=box.total_win
        total_loss+=box.total_loss
        mean_cost+=box.cost*box.spins_count
        
    return E/N, V/(N**2), N, total_win, total_loss, mean_cost/N

In [14]:
def get_boxes_stats(*l_of_b):
    E, V, N, total_win, total_loss, mean_cost = calc_boxes_stats(l_of_b)
    sum_ = total_win - total_loss
    mean_sum = sum_/N
    mean_win = total_win/N
    dist = norm(loc=E, scale=np.sqrt(V))
    p_better_res = dist.sf(mean_win)
    x = np.linspace(dist.ppf(0.005),dist.ppf(0.995), 100)
    plt.figure(figsize=(10,7))
    plt.plot(x, dist.pdf(x), label = 
            f'Theory mean profit: {round(E,2)}'+
            f'\nTheory variance: {round(V,2)}')
    plt.plot([mean_win], [0], 'o', label=f'Mean profit: {round(mean_win,2)}'+
             f'\nMean sum: {round(mean_sum,2)}'
             +f'\nBeter result probability: {round(100*p_better_res,2)}%')
    
    plt.plot([mean_cost], [0], 'o', label=f'Mean cost: {round(mean_cost,2)}'+
             '\n'+
            r'Greater profit probability: $\approx 10^{'
             +str(round(np.log(dist.sf(mean_cost))/np.log(10)+2))+
             '}$%')
    
    plt.ylabel(r'Probability density function f(x)', fontsize=15)
    plt.xlabel('Mean profit',  fontsize=15)
    plt.legend(loc=1)

In [15]:
def get_stats(*l_of_b):
    for i,b in enumerate(l_of_b):
        get_boxes_stats(b)
        plt.title(f'box #{i+1}', fontsize=20)
        
    get_boxes_stats(*l_of_b)
    plt.title(f'total', fontsize=20)
    plt.show()

In [23]:
def many_games(*l_of_b, count_of_spins = 1000, count_of_games = 200):
    results = []
    temp_results = []
    for game in range(count_of_games):
        for b in l_of_b:
            b.reset()
            temp_results.extend(b.spin(count_of_spins))
        results.append(np.mean(temp_results))
        
    get_boxes_stats(b)
    plt.hist(results, bins=50, normed=True)